# Importing libraries and loading dataset

In [1]:
!pip install transformers
!pip install emoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 17.8 MB/s eta 0:00:00


## Data manipulation libraries

In [2]:
import sys, os
import pandas as pd
import numpy as np
import json

import emoji
import contractions
import re

##  Scikit-learn packages

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

## Packages to define a BERT model

In [4]:
from transformers import TFBertModel, BertTokenizerFast, BertConfig

## Keras and TensorFlow packages

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense, Concatenate, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Layer, Attention

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading Dataset and list of emotions

In [8]:
# Importing train, validation and test datasets with preprocessed texts and labels
train_GE = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_clean.csv")
val_GE = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/val_clean.csv")
test_GE = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test_clean.csv")


In [ ]:
# Shape validation
print(train_GE.shape)
print(val_GE.shape)
print(test_GE.shape)

##  Loading emotion labels for GoEmotions taxonomy

In [9]:
with open("/content/drive/MyDrive/Colab Notebooks/emotions.txt", "r") as file:
    GE_taxonomy = file.read().split("\n")
GE_taxonomy.remove('neutral')
print("Emotions on GoEmotions taxonomy are : \n{}".format(GE_taxonomy))

print()

Emotions on GoEmotions taxonomy are : 
['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']



In [10]:
# Loading emotion labels for Ekman taxonomy
with open("/content/drive/MyDrive/Colab Notebooks/ekman_labels.txt", "r") as file:
    Ekman_taxonomy = file.read().split("\n")
Ekman_taxonomy.remove('neutral')
print("Emotions on Ekman taxonomy are : \n{}".format(Ekman_taxonomy))

Emotions on Ekman taxonomy are : 
['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


## Filtering out the 'Nautral' only samples

In [11]:
train_GE = train_GE.drop(columns=['neutral'])
val_GE = val_GE.drop(columns=['neutral'])
test_GE = test_GE.drop(columns=['neutral'])

## Removing all the samples that have been left without a label

In [12]:
# Removing samples with only 0 in their labels
train_GE = train_GE.loc[ train_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]
val_GE = val_GE.loc[ val_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]
test_GE = test_GE.loc[ test_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]

# Shape validation
print(train_GE.shape)
print(val_GE.shape)
print(test_GE.shape)

(30587, 28)
(3834, 28)
(3821, 28)


In [13]:
# Preview of data
display(train_GE.head(3))

,Clean_text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
2,why the fuck is bayless isoing,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,to make her feel threatened,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dirty southern wankers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modeling :BERT (Bidirectional Encoder Represnetation From Tranformers ) with attention mechanism

## Configuration of the Base model

### First of all let's define max_length variable.This variable sets a fixed length of sequences to be fed to our model.Therfore,sequence will be either truncated if larger than this value ,or completed using padding if smaller.To avoid truncating , we fix this value accirding to the largest sample of our data.

In [14]:
# Computing max length of samples
full_text = pd.concat([train_GE['Clean_text'], val_GE['Clean_text'], test_GE['Clean_text']])
max_length = full_text.apply(lambda x: len(x.split())).max()
max_length

48

## Importing BERT pre-trained model and tokenizer

In [15]:
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name, output_hidden_states=False)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

## Definition of the model Architecture  

Our model takes three inputs that result from tokenization:

*   `input_ids`: indices of input sequence tokens in the vocabulary
*   `token_ids`: Segment token indices to indicate first and second portions of the inputs.   0 for sentence A and 1 for sentence B
*   `attention mask`: Mask to avoid performing attention on padding token indices.  0 for masked and 1 for not masked



##  Define AttentionLayer

In [16]:

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # Inputs: [pooled_output, pooled_output]
        query, key = inputs

        # Compute attention scores
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = tf.nn.softmax(attention_scores, axis=-1)

        # Apply attention scores to value
        attention_output = tf.matmul(attention_scores, query)

        return attention_output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [17]:
def create_model_with_attention(nb_labels):
    # Load the MainLayer
    bert = transformer_model.layers[0]

    # Build the model inputs
    input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
    attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
    inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

    # Load the Transformers BERT model as a layer in a Keras model
    bert_model = bert(inputs)[1]
    dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
    pooled_output = dropout(bert_model, training=False)

    # Add Attention mechanism
    attention_output = AttentionLayer()([pooled_output, pooled_output])

    # Output layer
    emotion = Dense(units=nb_labels, activation="sigmoid", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(attention_output)

    # Combine it all in a model object
    model = Model(inputs=inputs, outputs=emotion, name='BERT_MultiLabel_with_Attention')

    return model




In [18]:
# Create a model instance with attention
model_with_attention = create_model_with_attention(27)

# Take a look at the model summary
model_with_attention.summary()

Model: "BERT_MultiLabel_with_Attention"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask (InputLayer  [(None, 48)]                 0         []                            
 )                                                                                                
                                                                                                  
 input_ids (InputLayer)      [(None, 48)]                 0         []                            
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1094822   ['attention_mask[0][0]',      
                             ngAndCrossAttentions(last_   40         'input_ids[0][0]']           
                             hidden_state=(None, 48, 76              

# Data Pre-Processing and Model training

## Tokenizing data

In [19]:
# Creating train, validation and test variables
X_train = train_GE['Clean_text']
y_train = train_GE.loc[:, GE_taxonomy].values.astype(float)

X_val = val_GE['Clean_text']
y_val = val_GE.loc[:, GE_taxonomy].values.astype(float)

X_test = test_GE['Clean_text']
y_test = test_GE.loc[:, GE_taxonomy].values.astype(float)

# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
val_token = tokenizer(
    text = X_val.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [20]:
# Creating BERT compatible inputs with Input Ids, attention masks and token Ids
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_ids': train_token['token_type_ids']}
val = {'input_ids': val_token['input_ids'], 'attention_mask': val_token['attention_mask'],'token_ids': val_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_ids': test_token['token_type_ids']}

In [21]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(16)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_val)).shuffle(len(val)).batch(16)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(16)

## Class weights for multi-lable and custom loss function

In [22]:
from sklearn.utils import compute_class_weight

# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced',
                                          classes=np.unique([0.,1.]),
                                          y = y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)
class_weights

array([[  0.57805118,   3.70302663],
       [  0.54119042,   6.56937285],
       [  0.52699862,   9.75973197],
       [  0.5439236 ,   6.1917004 ],
       [  0.55315032,   5.20364069],
       [  0.51842373,  14.06945722],
       [  0.52340943,  11.17945906],
       [  0.53857938,   6.98014605],
       [  0.5107026 ,  23.85881435],
       [  0.52164199,  12.05161545],
       [  0.53539296,   7.56355094],
       [  0.51330805,  19.28562421],
       [  0.50500264,  50.47359736],
       [  0.51434385,  17.92907386],
       [  0.50993631,  25.6602349 ],
       [  0.54766338,   5.74511645],
       [  0.50126188, 198.61688312],
       [  0.52491848,  10.5327135 ],
       [  0.53659521,   7.33149569],
       [  0.50269533,  93.25304878],
       [  0.52725298,   9.67330803],
       [  0.50182111, 137.77927928],
       [  0.51882824,  13.77792793],
       [  0.50251364,  99.95751634],
       [  0.50907063,  28.06146789],
       [  0.52265815,  11.53355958],
       [  0.51794967,  14.42783019]])

In [23]:
# Custom loss function for multilabel
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

## Model Training

In [27]:
# Set an optimizer
optimizer = Adam(
    learning_rate=3.e-05,
    )

In [28]:
# Compile the model
model_with_attention.compile(optimizer=optimizer, loss=loss)

In [30]:
# Set loss
loss = get_weighted_loss(class_weights)

# Compile the model
model_with_attention.compile(
    optimizer = optimizer,
    loss = loss
    )

# Train the model
history = model_with_attention.fit(train_tensor,
                                   epochs=8,
                                   validation_data=val_tensor)

Epoch 1/8


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['token_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


1912/1912 [==============================] - 417s 192ms/step - loss: 0.6284 - val_loss: 0.4615
Epoch 2/8
1912/1912 [==============================] - 363s 190ms/step - loss: 0.3811 - val_loss: 0.3370
Epoch 3/8
1912/1912 [==============================] - 363s 190ms/step - loss: 0.2733 - val_loss: 0.3139
Epoch 4/8
1912/1912 [==============================] - 364s 190ms/step - loss: 0.2100 - val_loss: 0.3290
Epoch 5/8
1912/1912 [==============================] - 364s 190ms/step - loss: 0.1661 - val_loss: 0.3692
Epoch 6/8
1912/1912 [==============================] - 364s 190ms/step - loss: 0.1363 - val_loss: 0.4244
Epoch 7/8
1912/1912 [==============================] - 363s 190ms/step - loss: 0.1148 - val_loss: 0.4596
Epoch 8/8
1912/1912 [==============================] - 362s 190ms/step - loss: 0.0943 - val_loss: 0.5207


## model Evaluation

In [31]:
# Save model weights
# model.save_weights('/content/drive/MyDrive/Go_emotion dataset/BERT Model')

In [33]:
# Making probability predictions on test data
y_pred_proba = model_with_attention.predict(test)

120/120 [==============================] - 19s 110ms/step


In [34]:
# from probabilities to labels using a given threshold
def proba_to_labels(y_pred_proba, threshold=0.8):

    y_pred_labels = np.zeros_like(y_pred_proba)

    for i in range(y_pred_proba.shape[0]):
        for j in range(y_pred_proba.shape[1]):
            if y_pred_proba[i][j] > threshold:
                y_pred_labels[i][j] = 1
            else:
                y_pred_labels[i][j] = 0

    return y_pred_labels

In [35]:
# Generate labels
y_pred_labels = proba_to_labels(y_pred_proba)

In [36]:
# Model evaluation function
def model_eval(y_true, y_pred_labels, emotions):

    # Defining variables
    precision = []
    recall = []
    f1 = []

    # Per emotion evaluation
    idx2emotion = {i: e for i, e in enumerate(emotions)}

    for i in range(len(emotions)):

        # Computing precision, recall and f1-score
        p, r, f1_score, _ = precision_recall_fscore_support(y_true[:, i], y_pred_labels[:, i], average="binary")

        # Append results in lists
        precision.append(round(p, 2))
        recall.append(round(r, 2))
        f1.append(round(f1_score, 2))

    # Macro evaluation
    macro_p, macro_r, macro_f1_score, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")

    # Append results in lists
    precision.append(round(macro_p, 2))
    recall.append(round(macro_r, 2))
    f1.append(round(macro_f1_score, 2))

    # Converting results to a dataframe
    df_results = pd.DataFrame({"Precision":precision, "Recall":recall, 'F1':f1})
    df_results.index = emotions+['MACRO-AVERAGE']

    return df_results

In [37]:
# Model evaluation
model_eval(y_test, y_pred_labels, GE_taxonomy)

,Precision,Recall,F1
admiration,0.71,0.67,0.69
amusement,0.79,0.89,0.84
anger,0.49,0.63,0.55
annoyance,0.36,0.58,0.44
approval,0.41,0.59,0.48
caring,0.36,0.63,0.46
confusion,0.45,0.68,0.54
curiosity,0.59,0.79,0.67
desire,0.57,0.55,0.56
disappointment,0.31,0.36,0.33


## Threshold Optimization

In [39]:
# Function that computes labels from probabilities and optimizes the threshold that maximizes f1-score
def proba_to_labels_opt(y_true, y_pred_proba):

    '''
    Inputs:
        y_true: Ground truth labels
        y_pred_proba: predicted probabilities

    Outputs :
        best_y_pred_labels: preticted labels associated with best threshold
        best_t: best threshold
        best_macro_f1: macro f1-score associated with predicted labels
    '''

    # range of possible thresholds
    thresholds = np.arange(0.7, 0.99, 0.01)

    # Computing threshold that maximizes macro f1-score
    best_y_pred_labels = np.zeros_like(y_pred_proba)
    best_t = 0
    best_macro_f1 = 0

    # Iterating through possible thresholds
    for t in thresholds:

        y_pred_labels = proba_to_labels(y_pred_proba, t)

        _, _, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")

        if macro_f1 > best_macro_f1:
            best_macro_f1 = macro_f1
            best_t = t
            best_y_pred_labels = y_pred_labels

    return best_y_pred_labels, best_t, best_macro_f1

In [40]:
# Compute label predictions and corresponding optimal thresholds
y_pred_labels_opt, threshold_opt, macro_f1_opt = proba_to_labels_opt(y_test, y_pred_proba)
print("The model's threshold is {}".format(threshold_opt))
print("The model's best macro-f1 is {}".format(macro_f1_opt))

The model's threshold is 0.9000000000000001
The model's best macro-f1 is 0.5380136154370843


In [41]:
# Model evaluation : Precision, Recall, F-score
model_eval(y_test, y_pred_labels_opt, GE_taxonomy)

,Precision,Recall,F1
admiration,0.75,0.60,0.67
amusement,0.82,0.86,0.84
anger,0.51,0.53,0.52
annoyance,0.41,0.43,0.42
approval,0.49,0.49,0.49
caring,0.42,0.59,0.49
confusion,0.52,0.56,0.53
curiosity,0.64,0.73,0.68
desire,0.66,0.48,0.56
disappointment,0.42,0.28,0.34


# Evaluation on Ekman Taxonomy

In [42]:
# Function thats maps predictions on GoEmotions taxonomy to Ekman taxonomy
def GE_to_Ekman(GE_labels):

    # Create a dataframe of GoEmotions labels
    df_GE = pd.DataFrame(GE_labels, columns=GE_taxonomy)

    # Create an empty dataframe of Ekman labels
    df_Ekman  = pd.DataFrame(np.zeros((len(GE_labels), len(Ekman_taxonomy))), columns=Ekman_taxonomy)

    for i in range(len(df_GE)):

        if df_GE.loc[i,['anger', 'annoyance', 'disapproval']].sum() >= 1:
            df_Ekman.loc[i,'anger'] = 1

        if df_GE.loc[i,'disgust'].sum() >= 1:
            df_Ekman.loc[i,'disgust'] = 1

        if df_GE.loc[i,['fear', 'nervousness']].sum() >= 1:
            df_Ekman.loc[i,'fear'] = 1

        if df_GE.loc[i,['joy', 'amusement', 'approval', 'excitement', 'gratitude',
                        'love', 'optimism', 'relief', 'pride', 'admiration', 'desire','caring']].sum() >= 1:
            df_Ekman.loc[i,'joy'] = 1

        if df_GE.loc[i,['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse']].sum() >= 1:
            df_Ekman.loc[i,'sadness'] = 1

        if df_GE.loc[i,['surprise', 'realization', 'confusion', 'curiosity']].sum() >= 1:
            df_Ekman.loc[i,'surprise'] = 1

    return df_Ekman.values

In [43]:
# Mapping GoEmotion labels to Ekman labels (true and predictions)
y_test_Ekman = GE_to_Ekman(y_test)
y_pred_labels_Ekman = GE_to_Ekman(y_pred_labels_opt)

# Evaluation
model_eval(y_test_Ekman, y_pred_labels_Ekman, Ekman_taxonomy)

,Precision,Recall,F1
anger,0.68,0.62,0.65
disgust,0.38,0.63,0.48
fear,0.58,0.73,0.65
joy,0.89,0.87,0.88
sadness,0.65,0.62,0.64
surprise,0.68,0.71,0.69
MACRO-AVERAGE,0.64,0.70,0.66


## Making Predictions

In [44]:
# Retrieving initial preprocessings
def preprocess_corpus(x):

    # Adding a space between words and punctation
    x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
    x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)

    # Demojize
    x = emoji.demojize(x)

    # Expand contraction
    x = contractions.fix(x)

    # Lower
    x = x.lower()

    #correct some acronyms/typos/abbreviations
    x = re.sub(r"lmao", "laughing my ass off", x)
    x = re.sub(r"amirite", "am i right", x)
    x = re.sub(r"\b(tho)\b", "though", x)
    x = re.sub(r"\b(ikr)\b", "i know right", x)
    x = re.sub(r"\b(ya|u)\b", "you", x)
    x = re.sub(r"\b(eu)\b", "europe", x)
    x = re.sub(r"\b(da)\b", "the", x)
    x = re.sub(r"\b(dat)\b", "that", x)
    x = re.sub(r"\b(dats)\b", "that is", x)
    x = re.sub(r"\b(cuz)\b", "because", x)
    x = re.sub(r"\b(fkn)\b", "fucking", x)
    x = re.sub(r"\b(tbh)\b", "to be honest", x)
    x = re.sub(r"\b(tbf)\b", "to be fair", x)
    x = re.sub(r"faux pas", "mistake", x)
    x = re.sub(r"\b(btw)\b", "by the way", x)
    x = re.sub(r"\b(bs)\b", "bullshit", x)
    x = re.sub(r"\b(kinda)\b", "kind of", x)
    x = re.sub(r"\b(bruh)\b", "bro", x)
    x = re.sub(r"\b(w/e)\b", "whatever", x)
    x = re.sub(r"\b(w/)\b", "with", x)
    x = re.sub(r"\b(w/o)\b", "without", x)
    x = re.sub(r"\b(doj)\b", "department of justice", x)

    # replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
    x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
    x = re.sub(r"\b(co+l+)\b", "cool", x)
    x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
    x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
    x = re.sub(r"\b(o+m+g+)\b", "omg", x)
    x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
    x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
    x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
    x = re.sub(r"\b(w+o+w+)\b", "wow", x)
    x = re.sub(r"\b(w+h+y+)\b", "why", x)
    x = re.sub(r"\b(s+o+)\b", "so", x)
    x = re.sub(r"\b(f)\b", "fuck", x)
    x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
    x = re.sub(r"\b(ofc)\b", "of course", x)
    x = re.sub(r"\b(the us)\b", "usa", x)
    x = re.sub(r"\b(gf)\b", "girlfriend", x)
    x = re.sub(r"\b(hr)\b", "human ressources", x)
    x = re.sub(r"\b(mh)\b", "mental health", x)
    x = re.sub(r"\b(idk)\b", "i do not know", x)
    x = re.sub(r"\b(gotcha)\b", "i got you", x)
    x = re.sub(r"\b(y+e+p+)\b", "yes", x)
    x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
    x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
    x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
    x = re.sub(r"\b(o+h+)\b", "oh", x)
    x = re.sub(r"\b(a+h+)\b", "ah", x)
    x = re.sub(r"\b(u+h+)\b", "uh", x)

    # Handling emojis
    x = re.sub(r"<3", " love ", x)
    x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
    x = re.sub(r":\)", " smiling_face ", x)
    x = re.sub(r"^_^", " smiling_face ", x)
    x = re.sub(r"\*_\*", " star_struck ", x)
    x = re.sub(r":\(", " frowning_face ", x)
    x = re.sub(r":\^\(", " frowning_face ", x)
    x = re.sub(r";\(", " frowning_face ", x)
    x = re.sub(r":\/",  " confused_face", x)
    x = re.sub(r";\)",  " wink", x)
    x = re.sub(r">__<",  " unamused ", x)
    x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
    x = re.sub(r"\b(n+a+h+)\b", "no", x)

    # Handling special cases of text
    x = re.sub(r"h a m b e r d e r s", "hamberders", x)
    x = re.sub(r"b e n", "ben", x)
    x = re.sub(r"s a t i r e", "satire", x)
    x = re.sub(r"y i k e s", "yikes", x)
    x = re.sub(r"s p o i l e r", "spoiler", x)
    x = re.sub(r"thankyou", "thank you", x)
    x = re.sub(r"a^r^o^o^o^o^o^o^o^n^d", "around", x)

    # Remove special characters and numbers replace by space + remove double space
    x = re.sub(r"\b([.]{3,})"," dots ", x)
    x = re.sub(r"[^A-Za-z!?_]+"," ", x)
    x = re.sub(r"\b([s])\b *","", x)
    x = re.sub(r" +"," ", x)
    x = x.strip()

    return x

In [45]:
def predict_samples(text_samples, model, threshold):

    # Text preprocessing and cleaning
    text_samples_clean = [preprocess_corpus(text) for text in text_samples]

    # Tokenizing train data
    samples_token = tokenizer(
        text = text_samples_clean,
        add_special_tokens = True,
        max_length = max_length,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'tf',
        return_token_type_ids = True,
        return_attention_mask = True,
        verbose = True,
    )

    # Preparing to feed the model
    samples = {'input_ids': samples_token['input_ids'],
               'attention_mask': samples_token['attention_mask'],
               'token_ids': samples_token['token_type_ids']
              }

    # Probability predictions
    samples_pred_proba = model.predict(samples)

    # Label prediction using threshold
    samples_pred_labels = proba_to_labels(samples_pred_proba)

    samples_pred_labels_df = pd.DataFrame(samples_pred_labels)
    samples_pred_labels_df = samples_pred_labels_df.apply(lambda x: [GE_taxonomy[i] for i in range(len(x)) if x[i]==1], axis=1)

    #return list(samples_pred_labels_df)
    return pd.DataFrame({"Text":text_samples, "Emotions":list(samples_pred_labels_df)})

In [2]:
# Predict samples
#predict_samples(["My favourite food is anything I didn't have to cook myself", "are you kiddin me ??!!", "red"], model_with_attention, threshold_opt)